# Class Weight Compution

In [3]:
import math
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
    
%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True) # don't use scientific notation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
# export_ds = '/media/scratch/berlukas/nuscenes'

# sem_clouds_filename = f"{export_ds}/sem_bkp/sem_classes_gt1.npy"
sem_clouds_filename = f"{export_ds}/sem_clouds2.npy"

print(f"Loading sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename).astype(int)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

In [5]:
# export_ds = '/media/scratch/berlukas/nuscenes/training'
export_ds = '/media/berlukas/Data/data/datasets/s2ae/cluster/training_kitti'
samples = os.listdir(export_ds)
sem_cloud_features = None
for sample in samples:
    sem_clouds_filename = f"{export_ds}/{sample}"    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename).astype(int)
    else:
        features = np.load(sem_clouds_filename).astype(int)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/berlukas/Data/data/datasets/s2ae/cluster/training_kitti/sem_clouds_kitti_bw50.npy
Shape of sem clouds is (18859, 2, 100, 100)


In [6]:
n_classes = 6
class_counts = [0] * n_classes
for cl in range(0, n_classes): 
    count = np.count_nonzero(sem_cloud_features[:,1,:,:] == cl)
    print(f'Computing occurrences for {cl}: {count}')
    class_counts[cl] = count

Computing occurrences for 0: 162899884
Computing occurrences for 1: 1648487
Computing occurrences for 2: 7624
Computing occurrences for 3: 12966034
Computing occurrences for 4: 4899243
Computing occurrences for 5: 6168728


In [7]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)
rates = class_counts / total_counts
print(f'Should sum to one: {np.sum(rates) == 1.0}')
print(f'\nRates are\n====================\n')
print(f'Vehicle: {rates[1]}')
print(f'Living: {rates[2]}')
print(f'Ground: {rates[3]}')
print(f'Manmade: {rates[4]}')
print(f'Vegetation: {rates[5]}')



inv_rates = 1-rates
# inv_rates[0] = 0
# inv_rates[8] = 0
# weights = np.round_((inv_rates) / np.sum(inv_rates), 20)

rates = rates[0:]
# weights = np.power(np.amax(rates) / rates, 1 / 3.0)
weights = 1 / rates

print(f'\nWeights are\n====================\n')
print(f'Ignore: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Living: {weights[2]}')
print(f'Ground: {weights[3]}')
print(f'Manmade: {weights[4]}')
print(f'Vegetation: {weights[5]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {weights}')

Counts are 
[162899884, 1648487, 7624, 12966034, 4899243, 6168728]
Should sum to one: False

Rates are

Vehicle: 0.008741115647701362
Living: 4.042632165014052e-05
Ground: 0.0687525001325627
Manmade: 0.02597827562437033
Vegetation: 0.03270973010233841

Weights are

Ignore: 1.157704937346671
Vehicle: 114.4018727475558
Living: 24736.35886673662
Ground: 14.544925611023386
Manmade: 38.493701986204805
Vegetation: 30.571942870556132


Weights sum to one: False (24935.529014889307)
weights: [    1.15770494   114.40187275 24736.35886674    14.54492561
    38.49370199    30.57194287]


In [8]:
mu = sem_cloud_features[:,0,:,:].mean()
sigma = sem_cloud_features[:,0,:,:].std()
print(f'We have a mean of {mu} and a std of {sigma}')

We have a mean of 0.7933497852484225 and a std of 5.9342539028087025
